# Enable Agent Tutorial Part 4: Plastic Detection RAG 챗봇

## 개요

플라스틱 탐지 Enable Agent와 Context Builder를 통합하여 대화형 인터페이스를 구현한다.

In [1]:
import json
import yaml
from pathlib import Path
from datetime import datetime
from typing import Dict, Any, List
from collections import defaultdict
from ultralytics import YOLO
from dotenv import load_dotenv
from openai import OpenAI
load_dotenv()
print("라이브러리 임포트 완료")

라이브러리 임포트 완료


In [2]:
# 이전 노트북의 클래스들 재정의
class PlasticDetectionAgent:
    def __init__(self, skill_path: str):
        with open(skill_path, 'r', encoding='utf-8') as f:
            self.skill = yaml.safe_load(f)
        model_path = self.skill['model_info']['model_path']
        self.model = YOLO(model_path) if Path(model_path).exists() else YOLO('yolov8n.pt')
        self.classes = self.skill['classes']
        print("PlasticDetectionAgent 초기화 완료")
    
    def detect(self, image_path: str, confidence_threshold: float = 0.25):
        results = self.model.predict(source=image_path, conf=confidence_threshold, verbose=False)
        detections = []
        for result in results:
            for box in result.boxes:
                cls_id = int(box.cls[0])
                if cls_id in self.classes:
                    info = self.classes[cls_id]
                    detections.append({
                        'class_id': cls_id, 'class_name': info['name'],
                        'confidence': float(box.conf[0]), 'bbox': box.xyxy[0].tolist(),
                        'recycle_code': info['recycle_code']
                    })
        return {'detections': detections, 'num_detections': len(detections), 'timestamp': datetime.now().isoformat()}
    
    def generate_tool_definition(self):
        return {
            "type": "function",
            "function": {
                "name": "detect_plastic",
                "description": "이미지에서 플라스틱 소재를 탐지한다",
                "parameters": {
                    "type": "object",
                    "properties": {"image_path": {"type": "string"}, "confidence_threshold": {"type": "number", "default": 0.25}},
                    "required": ["image_path"]
                }
            }
        }

class DetectionContextBuilder:
    def __init__(self, context_dir: str = 'context_store'):
        self.context_dir = Path(context_dir)
        self.context_dir.mkdir(exist_ok=True)
        self.log_file = self.context_dir / 'detection_logs.json'
        self.summary_file = self.context_dir / 'detection_summary.json'
        self.logs = self._load_logs()
        print(f"Context Builder 초기화 완료 (로그: {len(self.logs)}개)")
    
    def _load_logs(self):
        if self.log_file.exists():
            with open(self.log_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        return []
    
    def add_detection(self, result: Dict, image_path: str = None):
        self.logs.append({'detection_id': len(self.logs)+1, 'result': result, 'image_path': image_path, 'logged_at': datetime.now().isoformat()})
        with open(self.log_file, 'w', encoding='utf-8') as f:
            json.dump(self.logs, f, indent=2, ensure_ascii=False)
        self._update_summary()
    
    def _update_summary(self):
        if not self.logs: return
        class_stats = defaultdict(lambda: {'count': 0, 'confidences': []})
        for log in self.logs:
            for det in log['result']['detections']:
                class_stats[det['class_name']]['count'] += 1
                class_stats[det['class_name']]['confidences'].append(det['confidence'])
        for cls, stats in class_stats.items():
            stats['avg_confidence'] = sum(stats['confidences']) / len(stats['confidences'])
            del stats['confidences']
        summary = {'total_images': len(self.logs), 'total_detections': sum(l['result']['num_detections'] for l in self.logs),
                   'class_statistics': dict(class_stats), 'last_updated': datetime.now().isoformat()}
        with open(self.summary_file, 'w', encoding='utf-8') as f:
            json.dump(summary, f, indent=2, ensure_ascii=False)
    
    def get_summary(self):
        if self.summary_file.exists():
            with open(self.summary_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        return {}

print("클래스 정의 완료")

클래스 정의 완료


In [3]:
class PlasticDetectionChatbot:
    def __init__(self, agent: PlasticDetectionAgent, context_builder: DetectionContextBuilder):
        self.agent = agent
        self.context_builder = context_builder
        self.client = OpenAI()
        self.conversation_history = []
        print("Plastic Detection RAG 챗봇 초기화 완료")
    
    def _get_system_prompt(self):
        summary = self.context_builder.get_summary()
        return f"""당신은 플라스틱 재활용 분류 AI 어시스턴트다.

역할:
1. 이미지에서 플라스틱 소재를 탐지한다
2. 탐지 결과를 설명하고 재활용 방법을 안내한다
3. 과거 탐지 통계를 제공한다

현재 통계: {json.dumps(summary, ensure_ascii=False) if summary else '데이터 없음'}

클래스 정보:
- PET (#1): 음료수 병
- PS (#6): 스티로폼
- PP (#5): 요거트 용기
- PE (#2,#4): 비닐봉투

문장은 ~다로 끝낸다."""
    
    def chat(self, user_message: str, image_path: str = None) -> str:
        self.conversation_history.append({"role": "user", "content": user_message})
        messages = [{"role": "system", "content": self._get_system_prompt()}] + self.conversation_history
        tools = [self.agent.generate_tool_definition()]
        
        response = self.client.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools, tool_choice="auto")
        msg = response.choices[0].message
        
        if msg.tool_calls:
            for tc in msg.tool_calls:
                args = json.loads(tc.function.arguments)
                if image_path: args['image_path'] = image_path
                result = self.agent.detect(**args)
                self.context_builder.add_detection(result, args.get('image_path'))
                
                self.conversation_history.append({"role": "assistant", "content": None, "tool_calls": [tc]})
                self.conversation_history.append({"role": "tool", "tool_call_id": tc.id, "name": "detect_plastic",
                                                  "content": json.dumps(result, ensure_ascii=False)})
            
            final = self.client.chat.completions.create(model="gpt-4o-mini", messages=[{"role": "system", "content": self._get_system_prompt()}] + self.conversation_history)
            assistant_msg = final.choices[0].message.content
        else:
            assistant_msg = msg.content
        
        self.conversation_history.append({"role": "assistant", "content": assistant_msg})
        return assistant_msg
    
    def generate_report(self) -> str:
        summary = self.context_builder.get_summary()
        prompt = f"""플라스틱 탐지 분석 보고서를 작성해달라:\n{json.dumps(summary, indent=2, ensure_ascii=False)}\n\n내용: 1.현황 2.클래스별분석 3.권장사항. 문장은 ~다로 끝낸다."""
        response = self.client.chat.completions.create(model="gpt-4o", messages=[{"role": "user", "content": prompt}], max_tokens=1000)
        return response.choices[0].message.content
    
    def reset(self):
        self.conversation_history = []
        print("대화 초기화 완료")

print("PlasticDetectionChatbot 정의 완료")

PlasticDetectionChatbot 정의 완료


In [4]:
agent = PlasticDetectionAgent('skills/yolo_agent_skill.yaml')
context_builder = DetectionContextBuilder()
chatbot = PlasticDetectionChatbot(agent, context_builder)

PlasticDetectionAgent 초기화 완료
Context Builder 초기화 완료 (로그: 3개)
Plastic Detection RAG 챗봇 초기화 완료


In [5]:
def demo_chat(msg: str, img: str = None):
    print("="*60)
    print(f"사용자: {msg}")
    print("-"*60)
    response = chatbot.chat(msg, img)
    print(f"챗봇: {response}")
    print("="*60 + "\n")

In [6]:
demo_chat("안녕! 플라스틱 재활용에 대해 알려줘.")

사용자: 안녕! 플라스틱 재활용에 대해 알려줘.
------------------------------------------------------------
챗봇: 안녕! 플라스틱 재활용은 환경 보호와 자원 절약을 위해 매우 중요하다. 플라스틱은 여러 종류가 있으며, 각기 다른 재활용 방식이 필요하다. 대표적인 플라스틱 종류에는 PET, PS, PP, PE 등이 있다.

1. **PET (음료수 병)**: 가장 일반적으로 재활용되는 플라스틱으로, 재활용 후 섬유 제품이나 새로운 음료수 병으로 변환된다.
   
2. **PS (스티로폼)**: 스티로폼은 잘 분해되지 않지만, 일부 지역에서는 재활용이 가능하다. 보통 다른 재활용된 플라스틱과 혼합되어 사용된다.

3. **PP (요거트 용기)**: 재활용이 가능한 플라스틱으로, 종종 자동차 부품이나 새로운 용기로 변환된다.

4. **PE (비닐봉투)**: 재활용이 가능하나, 대부분의 재활용 시설에서는 다른 플라스틱과 혼합하기 때문에 특별히 수거해야 한다.

플라스틱을 재활용하려면 분리 배출을 철저히 해야 하며, 지역 재활용 규정을 확인해주의 것이 중요하다. 더 궁금한 사항이 있으면 언제든지 질문해달라.



In [7]:
test_images = list(Path('datasets/plastic/val/images').glob('*.jpg'))
if test_images:
    demo_chat("이 이미지에서 플라스틱 소재를 분석해줘.", str(test_images[0]))

사용자: 이 이미지에서 플라스틱 소재를 분석해줘.
------------------------------------------------------------
챗봇: 분석할 이미지를 업로드해주시면, 플라스틱 소재를 탐지하고 분석하겠다. 이미지를 제공해주기 바란다.



In [8]:
demo_chat("지금까지 탐지 통계를 알려줘.")

사용자: 지금까지 탐지 통계를 알려줘.
------------------------------------------------------------
챗봇: 현재까지의 탐지 통계는 다음과 같다:

- 총 이미지 수: 3
- 총 탐지 수: 14
- 클래스 통계:
  - PET (음료수 병): 5회 탐지, 평균 신뢰도 81.07%
  - PS (스티로폼): 3회 탐지, 평균 신뢰도 56.70%
  - PP (요거트 용기): 5회 탐지, 평균 신뢰도 66.37%
  - PE (비닐봉투): 1회 탐지, 평균 신뢰도 51.38%
  
마지막 업데이트: 2026년 1월 15일 13:06:44

더 궁금한 점이 있으면 물어보면 된다.



In [9]:
print("=== 분석 보고서 ===")
report = chatbot.generate_report()
print(report)

=== 분석 보고서 ===
## 플라스틱 탐지 분석 보고서

### 1. 현황
보고서에서는 총 3장의 이미지에서 총 14건의 플라스틱 탐지 결과가 보고되었다. 탐지된 플라스틱은 여러 종류로 분류되었으며, 각 클래스마다 검출된 개수와 평균 신뢰도를 바탕으로 분석이 수행되었다.

### 2. 클래스별 분석
- **PET (Polyethylene Terephthalate)**
  - 검출 개수: 5건
  - 평균 신뢰도: 0.8107
  - PET는 높은 신뢰도로 탐지되었으며, 플라스틱 중 가장 많이 검출된 클래스이다.
  
- **PS (Polystyrene)**
  - 검출 개수: 3건
  - 평균 신뢰도: 0.5670
  - PS는 상대적으로 낮은 신뢰도로 탐지됐다. PS에 대한 탐지 성능 개선이 필요하다.
  
- **PP (Polypropylene)**
  - 검출 개수: 5건
  - 평균 신뢰도: 0.6637
  - PP는 PET와 동일한 검출 건수를 보이나 신뢰도는 상대적으로 낮다.
  
- **PE (Polyethylene)**
  - 검출 개수: 1건
  - 평균 신뢰도: 0.5138
  - PE는 가장 적게 검출됐으며, 신뢰도 또한 가장 낮다. 이는 탐지 모델의 개선이 필요함을 시사한다.

### 3. 권장사항
1. **데이터 세트 확장**: 다양한 플라스틱 종류에 대한 충분한 데이터 수집이 필요하다. 특히 신뢰도와 검출 건수가 낮은 PS와 PE에 대한 추가적인 이미지 수집 및 학습이 요구된다.

2. **모델 개선**: 신뢰도가 낮은 플라스틱 클래스에 대해 탐지 모델의 성능을 향상시킬 필요가 있다. 이는 데이터 증강 또는 더 나은 탐지 알고리즘 적용을 통해 수행할 수 있다.

3. **정기적 업데이트**: 탐지 모델의 성능을 유지하고 개선하기 위해 주기적인 업데이트와 검증이 필요하다. 이를 통해 변화하는 플라스틱 사용 환경에 대응할 수 있을 것이다.

보고서는 2026년 1월 15일에 마지막으로 업데이트되었다. 이러한 권장사항을 바탕으로 지속적인